# My attempt to replicate this:

https://medium.com/emergent-future/simple-reinforcement-learning-with-tensorflow-part-0-q-learning-with-tables-and-neural-networks-d195264329d0

As interested in the creation of the environment as I am building and training the agent, so going to build both. 

S - start<br>
O - floor<br>
D - doom<br>
G - goal

## Preview

<p align="center"><img src="./q-learning-gif.gif"/></p>

---

###### Update 2018-01-18

It appears to work. I had to do a bit of a hack to get the Q-table working properly. I'd go back to fix it but I have a bunch of other projects in the queue, so moving on. 

In [74]:
import numpy as np
import sys
import time
from IPython.display import clear_output # for the hacky render piece

In [75]:
## create environment
# S - start
# O - floor
# D - doom
# G - goal

# note: not going to do the random piece that throws off the agent
# like openai does. could, but not going to at the moment

test_env = np.array([

    ['S', 'O', 'D'],
    ['O', 'D', 'O'],
    ['O', 'D', 'O'],
    ['O', 'O', 'O'],
    ['D', 'G', 'D']
   
])

actions = {
           "up":1,
           "right":2,
           "down":3,
           "left":4
          }

In [76]:
# create the main func that will be doing the acting

def actor(action,
          state,
          env = test_env):

    if action == 1:
        
        update = state[0]-1
        
        if update < 0:
            pass
        else:
            state[0] = update

    elif action == 2: 
        
        update = state[1]+1
        
        if update > 2:
            pass
        else:
            state[1] = update

    elif action == 3:
        
        update = state[0]+1
       
        if update < 0:
            pass
        else:
            state[0] = update

    else:
        
        update = state[1]-1

        if update < 0:
            pass
        else:
            state[1] = update
            
    # return vals        

    i0 = state[0]
    i1 = state[1]

    updated_state = env[i0][i1]
    
    if updated_state == "G":
        
        DONE = True
        reward = 1
    
    elif updated_state == "D":
        
        DONE = True
        reward = 0
        
    else:
        
        DONE = False
        reward = 0
        
    return state,DONE,reward

In [77]:
# this will show the actor moving the env for each episode

def displayer(state,
              n_episode,
              env=test_env):
    
    copy_env = env.copy()
    
    copy_env[state[0]][state[1]] = "*" 

    env_string = '\n'.join([''.join(['{:4}'.format(item) for item in row]) for row in copy_env])

    sys.stdout.write("Episode "+str(n_episode)+'\n\r'+env_string)
    time.sleep(.05)
    clear_output(wait=True)

In [78]:
# define execution vars

start_state = [0,0]
all_rewards = {} # key (n_episode) : value (array of vals to take the max of)

n_episodes = 250

# the below is ugly. I could avoid it, but I'd have to redo the way I create the env.
# going with it for now, maybe will return (probs not)

states_to_state = {
    
       '0':[0,0],
       '1':[0,1],
       '2':[0,2],
       '3':[1,0],
       '4':[1,1],
       '5':[1,2],
       '6':[2,0],
       '7':[2,1],
       '8':[2,2],
       '9':[3,0],
       '10':[3,1],
       '11':[3,2],
       '12':[4,0],
       '13':[4,1],
       '14':[4,2]

}

In [79]:
### the Q-learning/table vars

# in essence: for each state we need to get the value of each action
# as runs start to become more successful, then values start to become more of a signal

#Initialize table with all zeros

QTABLE = np.zeros([test_env.shape[0]*test_env.shape[1],len(actions)])

lr = .8
y = .95

In [80]:
for e in range(1,n_episodes):
    
    n_step = 0
    running_states = [start_state]
    
    while n_step <99: # don't let agent go pass 99 steps per episode

        step_rewards = [] # for collecting all rewards
        
        # convert raw state coordinates to corresponding Q-table number
        sN = int(list(states_to_state.keys())[list(states_to_state.values()).index(running_states[n_step])])

        a = np.argmax(QTABLE[sN,:] + np.random.randn(1,len(actions))*(1./(e+1))) # select action

        ### main func
        s,DONE,reward = actor(

            action = a,
            state = list(running_states[n_step])

        )
        
        sN1 = int(list(states_to_state.keys())[list(states_to_state.values()).index(s)]) 

        # display grid
        #displayer(state=s,n_episode=e)
        
        # update Q-table
        QTABLE[sN,a] = QTABLE[sN,a] + lr*(reward + y*np.max(QTABLE[sN1,:]) - QTABLE[sN,a]) 
        
        # update latest state
        running_states.append(s)

        # update latest reward
        step_rewards.append(reward)

        # find reward per episode (0 or 1)
        all_rewards[e] = max(step_rewards)

        # increment step
        n_step += 1

        if DONE:

            break